In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Load dataset
data = pd.read_csv('../notebooks/yamnet_balanced.csv')

# Split features and labels
X = data.iloc[:, :-1].values  # All feature columns
y = data.iloc[:, -1].values   # Label column

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
def create_data_loader(X, y, batch_size):
    dataset = TensorDataset(X, y)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

batch_size = 256  # Increased batch size
train_loader = create_data_loader(X_train_tensor, y_train_tensor, batch_size)
test_loader = create_data_loader(X_test_tensor, y_test_tensor, batch_size)

# Define the FNN model
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(FeedForwardNN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 1024),  # First hidden layer
            nn.BatchNorm1d(1024),       # Batch normalization
            nn.LeakyReLU(),
            nn.Dropout(0.3),            # Dropout for regularization
            nn.Linear(1024, 512),       # Second hidden layer
            nn.BatchNorm1d(512),        # Batch normalization
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),        # Third hidden layer
            nn.BatchNorm1d(256),        # Batch normalization
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, output_dim)  # Output layer
        )


    def forward(self, x):
        return self.fc(x)

# Initialize the model
input_dim = X_train.shape[1]
output_dim = len(np.unique(y))
model = FeedForwardNN(input_dim, output_dim)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.00005, weight_decay=1e-5)  # Switched to AdamW optimizer with lower learning rate

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

# Early stopping class
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# Training loop

def train_model(model, train_loader, criterion, optimizer, scheduler, epochs=200):
    model.train()
    early_stopping = EarlyStopping(patience=15, min_delta=0.005)

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)
        scheduler.step(avg_loss)  # Update learning rate based on loss
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

        # Check early stopping
        early_stopping(avg_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

# Train the model
train_model(model, train_loader, criterion, optimizer, scheduler, epochs=100)

# Evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

# Evaluate
evaluate_model(model, test_loader)


Epoch 1/100, Loss: 1.8116
Epoch 2/100, Loss: 1.5484
Epoch 3/100, Loss: 1.4151
Epoch 4/100, Loss: 1.3221
Epoch 5/100, Loss: 1.2522
Epoch 6/100, Loss: 1.1936
Epoch 7/100, Loss: 1.1449
Epoch 8/100, Loss: 1.1002
Epoch 9/100, Loss: 1.0726
Epoch 10/100, Loss: 1.0450
Epoch 11/100, Loss: 1.0089
Epoch 12/100, Loss: 0.9889
Epoch 13/100, Loss: 0.9625
Epoch 14/100, Loss: 0.9409
Epoch 15/100, Loss: 0.9223
Epoch 16/100, Loss: 0.9052
Epoch 17/100, Loss: 0.8906
Epoch 18/100, Loss: 0.8602
Epoch 19/100, Loss: 0.8649
Epoch 20/100, Loss: 0.8395
Epoch 21/100, Loss: 0.8228
Epoch 22/100, Loss: 0.8150
Epoch 23/100, Loss: 0.7974
Epoch 24/100, Loss: 0.7798
Epoch 25/100, Loss: 0.7606
Epoch 26/100, Loss: 0.7624
Epoch 27/100, Loss: 0.7387
Epoch 28/100, Loss: 0.7316
Epoch 29/100, Loss: 0.7164
Epoch 30/100, Loss: 0.7089
Epoch 31/100, Loss: 0.7068
Epoch 32/100, Loss: 0.6886
Epoch 33/100, Loss: 0.6715
Epoch 34/100, Loss: 0.6610
Epoch 35/100, Loss: 0.6614
Epoch 36/100, Loss: 0.6545
Epoch 37/100, Loss: 0.6414
Epoch 38/1